In [2]:

%%bash
cd ../data/all_marks

# Get statistics for HiMoRNA and FANTOM6 genes (and their overlap, i.e. Genya' confusion matrix counts)
head -n 1  H3K27ac/our_fantom_fantom_aso_genes_association.tsv
grep 'RP11-398K22.12'  H3K27ac/our_fantom_fantom_aso_genes_association.tsv

head -n 1  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv
grep 'RP11-398K22.12'  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv


head -n 1  H3K27ac/our_fantom_fantom_aso_genes_association.tsv
grep 'ASO_G0229852_03'  H3K27ac/our_fantom_fantom_aso_genes_association.tsv

head -n 1  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv
grep 'ASO_G0229852_03'  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv

head -n 1  H3K27ac/our_fantom_fantom_aso_genes_confusion_matrix.tsv
grep 'ASO_G0229852_03'  H3K27ac/our_fantom_fantom_aso_genes_confusion_matrix.tsv


lncRNAId	lncRNAName	our	our_p	our_m	fantom	fantom_p	fantom_m	intersection	mm_intersection	mp_intersection	pm_intersection	pp_intersection
ENSG00000229852_ASO_G0229852_05	RP11-398K22.12	1376	884	502	112	15	97	18	2	2	14	0
ENSG00000229852_ASO_G0229852_03	RP11-398K22.12	1376	884	502	1028	432	596	86	20	14	43	10
ENSG00000229852_ASO_G0229852_AD_01	RP11-398K22.12	1376	884	502	3	2	1	1	0	0	1	0
lncRNAId	lncRNAName	pvalue	mm_pvalue	mp_pvalue	pm_pvalue	pp_pvalue
ENSG00000229852_ASO_G0229852_05	RP11-398K22.12	0.00015872632353190242	1.0	0.18784005572006668	3.903002885970757e-05	1.0
ENSG00000229852_ASO_G0229852_03	RP11-398K22.12	0.000494804906776989	0.07098412305104422	0.299873488952345	6.38947887013619e-05	0.8172112452872375
lncRNAId	lncRNAName	our	our_p	our_m	fantom	fantom_p	fantom_m	intersection	mm_intersection	mp_intersection	pm_intersection	pp_intersection
ENSG00000229852_ASO_G0229852_03	RP11-398K22.12	1376	884	502	1028	432	596	86	20	14	43	10
lncRNAId	lncRNAName	pvalue	mm_pvalue	mp_pvalue	pm_pval

CalledProcessError: Command 'b"cd ../data/all_marks\n\n# Get statistics for HiMoRNA and FANTOM6 genes (and their overlap, i.e. Genya' confusion matrix counts)\nhead -n 1  H3K27ac/our_fantom_fantom_aso_genes_association.tsv\ngrep 'RP11-398K22.12'  H3K27ac/our_fantom_fantom_aso_genes_association.tsv\n\nhead -n 1  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv\ngrep 'RP11-398K22.12'  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv\n\n\nhead -n 1  H3K27ac/our_fantom_fantom_aso_genes_association.tsv\ngrep 'ASO_G0229852_03'  H3K27ac/our_fantom_fantom_aso_genes_association.tsv\n\nhead -n 1  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv\ngrep 'ASO_G0229852_03'  H3K27ac/our_fantom_fantom_aso_genes_association_pvalues.tsv\n\nhead -n 1  H3K27ac/our_fantom_fantom_aso_genes_confusion_matrix.tsv\ngrep 'ASO_G0229852_03'  H3K27ac/our_fantom_fantom_aso_genes_confusion_matrix.tsv\n"' returned non-zero exit status 1.

In [5]:
makeConfusionMatrix(target[0], annotation_prefix="fantom_aso", common_genes_set_length=24701)

350 ../data/ASO_names.info


In [3]:
import scipy.stats as stats
TP = 86
FP = 1290
FN = 942
TN = 22383
_, pv = stats.fisher_exact([[TP, FP], [FN, TN]])
pv

0.00016493496891960925

In [7]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

In [9]:
def makeExactFisherTest(target, annotation_prefix="", our_fantom_file_name=""):
    annotation_prefix = "_" + annotation_prefix if annotation_prefix else ""
    confuion_df = pd.read_csv("../data/all_marks/" + target + "/" + our_fantom_file_name + annotation_prefix + "_genes_confusion_matrix.tsv", sep="\t") 
    df = pd.read_csv("../data/all_marks/" + target + "/" + our_fantom_file_name + annotation_prefix + "_genes_association.tsv", sep="\t") 
    
    pvalue = []
    mm_pvalue = []
    mp_pvalue = []
    pm_pvalue = []
    pp_pvalue = []
    for index, row in confuion_df.iterrows():
        _, pv = stats.fisher_exact([[row['TP'], row['FP']], [row['FN'], row['TN']]])
        pvalue.append(pv)

        _, mm_pv = stats.fisher_exact([[row['mm_TP'], row['mm_FP']], [row['mm_FN'], row['mm_TN']]])
        mm_pvalue.append(mm_pv)

        _, mp_pv = stats.fisher_exact([[row['mp_TP'], row['mp_FP']], [row['mp_FN'], row['mp_TN']]])
        mp_pvalue.append(mp_pv)

        _, pm_pv = stats.fisher_exact([[row['pm_TP'], row['pm_FP']], [row['pm_FN'], row['pm_TN']]])
        pm_pvalue.append(pm_pv)

        _, pp_pv = stats.fisher_exact([[row['pp_TP'], row['pp_FP']], [row['pp_FN'], row['pp_TN']]])  
        pp_pvalue.append(pp_pv)
        
    adjusted_pvalue = multipletests(pvalue, alpha=0.05, method='fdr_bh')
    adjusted_mm_pvalue = multipletests(mm_pvalue, alpha=0.05, method='fdr_bh')
    adjusted_mp_pvalue = multipletests(mp_pvalue, alpha=0.05, method='fdr_bh')
    adjusted_pm_pvalue = multipletests(pm_pvalue, alpha=0.05, method='fdr_bh')
    adjusted_pp_pvalue = multipletests(pp_pvalue, alpha=0.05, method='fdr_bh')
    
    pvalue_df = pd.DataFrame({'pvalue': adjusted_pvalue[1], 'mm_pvalue': adjusted_mm_pvalue[1], 
                              'mp_pvalue': adjusted_mp_pvalue[1], 'pm_pvalue': adjusted_pm_pvalue[1], 
                              'pp_pvalue': adjusted_pp_pvalue[1]})
    
    df_pv = pd.concat([df[['lncRNAId', 'lncRNAName']], pvalue_df], axis = 1)
    df_pv = df_pv[df_pv.apply(lambda row: len([p for p in row.tolist()[3:] if float(p) < 0.05]) != 0, axis=1)]
    # df_pv.to_csv("../all_marks/" + target + "/" + our_fantom_file_name + annotation_prefix + "_genes_association_pvalues.tsv", sep="\t", index=None)
    
    return df_pv

In [4]:
# targets = [("H3K27ac", "_narrow"), ("H3K36me3", ""), 
#            ("H3K4me1", "_narrow"), ("H3K4me2", "_narrow"), ("H3K4me3", "_narrow"), ("H3K79me2", ""), 
#            ("H3K9ac", "_narrow"), ("H3K9me3", ""), ("H4K20me1", ""), ("H3K27me3", ""), ("methylation", "")]

In [13]:
# makeConfusionMatrix(target[0], annotation_prefix="fantom_aso", common_genes_set_length=24701)
print("makeExactTARIDFisherTest...")
df_pv = makeExactFisherTest("H3K27ac", 'fantom_aso', our_fantom_file_name="our_fantom")

makeExactTARIDFisherTest...


In [16]:
df_pv[df_pv.lncRNAName == "RP11-398K22.12"]

,lncRNAId,lncRNAName,pvalue,mm_pvalue,mp_pvalue,pm_pvalue,pp_pvalue
19,ENSG00000229852_ASO_G0229852_05,RP11-398K22.12,0.000159,1.000000,0.187840,0.000039,1.000000
31,ENSG00000229852_ASO_G0229852_03,RP11-398K22.12,0.000495,0.070984,0.299873,0.000064,0.817211
